In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
from pandas import *
from matplotlib.pyplot import *
%matplotlib inline 
# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv' )

In [ ]:
df

In [ ]:
df.dropna(axis = 1,inplace = True)

In [ ]:
df

In [ ]:
df['Class'] = df['v1']

In [ ]:
df.drop('v1',axis = 1 , inplace = True)

In [ ]:
df

In [ ]:
df['Class'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df['v2'].count()

In [ ]:
import re 
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
corpus = []
for i in range(df['v2'].count()):
    review = re.sub('[^a-zA-Z]',' ',df['v2'][i])
    review = review.lower()
    review = review.split()
    als = stopwords.words('english')
    als.remove('not')
    l = WordNetLemmatizer()
    review = [l.lemmatize(word) for word in review if word not in set(als)]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features = 5000)
x = cv.fit_transform(corpus).toarray()
y = df.iloc[:,-1].values

In [ ]:
y

In [ ]:
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
y = lab.fit_transform(y)

In [ ]:
y

In [ ]:
len(x[1])

In [ ]:
from imblearn.combine import SMOTETomek
from collections import Counter

In [ ]:
s = SMOTETomek(random_state = 0)
x_res , y_res = s.fit_resample(x,y)

In [ ]:
Counter(y_res)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_res,y_res,test_size = 0.2,random_state = 0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB


In [ ]:
models = [LogisticRegression(random_state = 0) , KNeighborsClassifier(n_neighbors = 5),SVC(kernel = 'rbf'),SVC(kernel = 'linear'),MultinomialNB(),GaussianNB(),
         DecisionTreeClassifier(criterion = 'entropy'),RandomForestClassifier(n_estimators = 100)]

In [ ]:
from sklearn.metrics import confusion_matrix , accuracy_score

In [ ]:
for i in models:
    reg = i
    
    reg.fit(x_train,y_train)
    
    y_ans = reg.predict(x_test)
    
    l = str(i)
    l = l.split('(')
    print(l[0])
    print(' ')
    print(confusion_matrix(y_test,y_ans))
    tn, fp, fn, tp = confusion_matrix(y_test, y_ans).ravel()
    fpr = (fp)/(fp+tn)
    print('False Positive Rate : {}'.format(fpr))
    

    print(accuracy_score(y_test,y_ans))
    

    print()

# * From the above results we can see that the False Positive Rate is very less when we used Multinomial Naive Bayes model . 
# * Here , we have to look which model gives us the lowest FPR because imagine we have classified a legitimate mail as spam ? Is it ok ? No , this will be a serious error . 
# * It is ok to classify spam mails as legitimate because this won't bring in much trouble for the user . Because for the user Legitimate mails are more important . 